## Adding libraries

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, confusion_matrix

import tensorflow as tf
tfl = tf.keras.layers
tfr = tf.keras.regularizers
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import keras
from keras import layers
from keras.applications import EfficientNetB2, Xception, ResNet101

In [ ]:
num_classes = 1
batch_size = 32
height = width = 600

## Dataset Preparation

In [ ]:
damaged_path = ""
durable_path = ""

def load_data(path, label):
    X = []
    y = []
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = Image.open(img_path)
        if img is not None:
            img = img.resize((height, width), Image.LANCZOS).convert('RGB')
            X.append(img)
            y.append(label)
    return X, y

X_hasarli, y_hasarli = load_data(damaged_path, 1)
X_hasarsiz, y_hasarsiz = load_data(durable_path, 0)

X = np.array(X_hasarli + X_hasarsiz)
y = np.array(y_hasarli + y_hasarsiz)

In [ ]:
shuffle_indices = np.random.permutation(len(X))
X_shuffled = X[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_shuffled, y_shuffled, test_size=0.25, random_state=42)
print("Number of train images: ", len(X_train), " Number of test images: ", len(X_test))

## Drawing Operations

In [ ]:
def plot_hist(hist):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

    ax1.plot(hist.history["accuracy"], label="Training Accuracy")
    ax1.plot(hist.history["val_accuracy"], label="Validation Accuracy")
    ax1.set_title("Model Accuracy")
    ax1.set_ylabel("Accuracy")
    ax1.set_xlabel("Epoch")
    ax1.legend()

    ax2.plot(hist.history["loss"], label="Training Loss")
    ax2.plot(hist.history["val_loss"], label="Validation Loss")
    ax2.set_title("Model Loss")
    ax2.set_ylabel("Loss")
    ax2.set_xlabel("Epoch")
    ax2.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def showResults(test, pred):
    target_names = ['positive', 'negative']
    print(classification_report(test, pred, target_names=target_names))
    accuracy = accuracy_score(test, pred)
    precision=precision_score(test, pred, average='weighted')
    f1Score=f1_score(test, pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(test, pred)
    print(cm)

## Finding the Heat Map

In [ ]:
def visualize_gradcam(model, input_image, alpha=0.5, display=True):
    assert 0 < alpha < 1, "Alpha value must be between 0 and 1"

    conv_layer = next(layer for layer in reversed(model.layers) if isinstance(layer, tf.keras.layers.Conv2D))
    target_conv_layer = model.get_layer(conv_layer.name)

    processed_img = np.asarray(input_image)
    if processed_img.ndim == 2:
        processed_img = np.expand_dims(processed_img, axis=-1)
    processed_img = np.expand_dims(processed_img, axis=0)

    prediction = model.predict(processed_img)
    print("Prediction Result:", prediction)
    print("Status:", "Intact" if prediction < 0.5 else "Damaged")
    predicted_class = np.argmax(prediction)

    with tf.GradientTape() as tape:
        grad_model = Model(inputs=model.input, outputs=[target_conv_layer.output, model.output])
        conv_output, predictions = grad_model(processed_img)
        loss = predictions[:, predicted_class]

    grads = tape.gradient(loss, conv_output)
    pooled_grads = tf.reduce_mean(grads[0], axis=(0, 1))

    heatmap = np.zeros(conv_output.shape[1:3], dtype=np.float32)
    for i, weight in enumerate(pooled_grads):
        heatmap += weight * conv_output[0, :, :, i]

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.resize(heatmap, (processed_img.shape[2], processed_img.shape[1]))

    colored_map = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    original_image = np.uint8(255 * (processed_img[0] - processed_img.min()) / (processed_img.max() - processed_img.min()))
    blended_image = np.uint8(original_image * alpha + colored_map * (1 - alpha))

    if display:
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        ax[0].imshow(original_image)
        ax[0].axis("off")
        ax[0].set_title("Original Image")

        ax[1].imshow(blended_image)
        ax[1].axis("off")
        ax[1].set_title("Grad-CAM Map")

        plt.show()
    else:
        return blended_image

## Creating the Transfer Learning Model

In [ ]:
def build_model(num_classes,drop_prob=0.4):
    inputs = layers.Input(shape=(height, width, 3), name='input_layer')

    # Model
    #model = EfficientNetB2(input_tensor=x, weights="imagenet",include_top=False)
    #model = ResNet101(input_tensor=inputs, weights="imagenet",include_top=False)
    model = Xception(input_tensor=inputs, weights="imagenet", include_top=False)
    model.trainable = False

    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.Dropout(drop_prob)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(drop_prob)(x)

    x = layers.Dense(num_classes, activation="sigmoid", name='output_layer')(x)
    outputs = x

    optimizer = keras.optimizers.Adam()
    model = keras.Model(inputs, outputs, name=model.name)
    model.compile(
        optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=["accuracy"]
    )
    return model

## Education Phase

In [ ]:
checkpoint_path = "model_checkpoint.h5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True)

In [ ]:
model = build_model(num_classes,drop_prob=0.4)
hist = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=15)

## Educational evaluation

In [ ]:
plot_hist(hist)

In [ ]:
yPred = model.predict(X_test)
yTest = to_categorical(y_test)
pred = np.argmax(yPred, axis=1)
test = np.argmax(yTest, axis=1)
showResults(test, pred)

## Testting

In [ ]:
image_test_path = ""
img = Image.open(image_test_path)
img = img.resize((height,width))
img = np.array(img)
img = img/255.0
visualize_gradcam(model,img)